In [ ]:
# preprocessing of a dataset

import pandas as pd
import numpy as np


df=pd.read_csv("data1.csv")
df.shape
# df.isnull().sum()
# df.dropna(inplace=True)
# df.duplicated().sum()

In [ ]:
# creating input and output data

features = df.drop("Churn", axis=1)
target = df["Churn"]

In [ ]:
# Label Encoding the target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(target)

In [ ]:
#   datatype of features
features.dtypes

In [ ]:
#  converting object datatype to category
categorical_columns=[]
for col in features.columns:
    if features[col].dtype != 'int64':
        categorical_columns.append(col)

# converting object datatype to category
for col in categorical_columns:
    features[col] = features[col].astype('category')

features.dtypes

In [ ]:
filter_col=list(features.select_dtypes(exclude=['category']).columns)
filter_col

In [ ]:
# one_hot encoding
features = pd.get_dummies(features)
features.dtypes

In [47]:
# scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for col in filter_col:
    features[col] = scaler.fit_transform(features[[col]])

# features.head(10)